<div style="display: flex; align-items: center;">
  <img src="https://raw.githubusercontent.com/bartczernicki/DecisionIntelligence.GenAI.Workshop/main/Images/DecisionIntelligenceSemanticKernelLogo.png" width="60px" style="margin-right: 10px;">
  <span style="font-size: 1.5em; font-weight: bold;">Semantic Kernel - Decisions with AI Agent Personas</span>
</div>

Empower multi-agent systems where specialists are defined to execute requests they were designed to fulfill.  

### Step 1 - Initialize Configuration Builder & Build the Semantic Kernel Orchestration 

Execute the next two cells to:
* Use the Configuration Builder to load the API secrets.  
* Use the API configuration to build the Semantic Kernel orchestrator.

In [1]:
// Import the required NuGet configuration packages
#r "nuget: Microsoft.Extensions.Configuration, 9.0.8"
#r "nuget: Microsoft.Extensions.Configuration.Json, 9.0.8"

using Microsoft.Extensions.Configuration.Json;
using Microsoft.Extensions.Configuration;
using System.IO;

// Load the configuration settings from the local.settings.json and secrets.settings.json files
// The secrets.settings.json file is used to store sensitive information such as API keys
var configurationBuilder = new ConfigurationBuilder()
    .SetBasePath(Directory.GetCurrentDirectory())
    .AddJsonFile("local.settings.json", optional: true, reloadOnChange: true)
    .AddJsonFile("secrets.settings.json", optional: true, reloadOnChange: true);
var config = configurationBuilder.Build();

// IMPORTANT: You ONLY NEED either Azure OpenAI or OpenAI connectiopn info, not both.
// Azure OpenAI Connection Info
var azureOpenAIEndpoint = config["AzureOpenAI:Endpoint"];
var azureOpenAIAPIKey = config["AzureOpenAI:APIKey"];
var azureOpenAIModelDeploymentName = config["AzureOpenAI:ModelDeploymentName"];
var azureOpenAIReasoningEndpoint = config["AzureOpenAI:ReasoningEndpoint"];
var azureOpenAIReasoningAPIKey = config["AzureOpenAI:ReasoningAPIKey"];
var azureOpenAIReasoningModelDeploymentName = config["AzureOpenAI:ReasoningModelDeploymentName"];
// OpenAI Connection Info 
var openAIAPIKey = config["OpenAI:APIKey"];
var openAIModelId = config["OpenAI:ModelId"];

Installed Packages Microsoft.Extensions.Configuration, 9.0.8 Microsoft.Extensions.Configuration.Json, 9.0.8

In [2]:
// Import the Semantic Kernel NuGet package
#r "nuget: Microsoft.SemanticKernel, 1.62" 
#r "nuget: Microsoft.SemanticKernel.Agents.Core, 1.62"

using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.Agents; // Agents package needed for Agents support in the Semantic Kernel
using Microsoft.SemanticKernel.ChatCompletion;
using Microsoft.SemanticKernel.Connectors.OpenAI;

// Create SK Kernel with both general purpose and reasoning AI model services 
var semanticKernel = Kernel.CreateBuilder()
    .AddAzureOpenAIChatCompletion(
        deploymentName: azureOpenAIModelDeploymentName!,
        endpoint: azureOpenAIEndpoint!,
        apiKey: azureOpenAIAPIKey!,
        serviceId: "GeneralAI")
    .AddAzureOpenAIChatCompletion(
        // This is optional, but you can add a second model for fallback or other purposes
        deploymentName: azureOpenAIReasoningModelDeploymentName!,
        endpoint: azureOpenAIReasoningEndpoint!,
        apiKey: azureOpenAIReasoningAPIKey!,
        serviceId: "ReasoningAI") // Fallback service id, if needed
    .Build();

Installed Packages Microsoft.SemanticKernel, 1.62.0 Microsoft.SemanticKernel.Agents.Core, 1.62.0

### Step 2 - Creating an AI Agent Decision Persona

Agent personas can leverage the Chat Completion service from the Semantic Kernel Agents package. Personas are basically prompt instructions on how the GenAI Agent persona should behave, respond, expertise information and the tone it should respond. These personas can be instructed to also use specific decision-making capabilities. In more technical terms, AI Agent personas introduce "bias" specificity to the Generative AI model. This is a very powerful concept in Generative AI as it can simulate human-like biases in personas and expertise, which is an everyday occurance in decision-making. 

> Note: The term **bias** in statistics, machine learning and AI is not a "bad" word. Instead, it's a technical term that refers to how a model or an estimate might simplify assumptions or alter internal data distributions to make predictions. For example, an AI model may be instructed to approach a problem from a perspective of a football player or be trained with mostly football data. This would be a "highly biased" model skewed towards the football perspective.  

In this example, the agent will take on the persona of **Dwight Eisenhower**. What does Dwight Eisenhower have todo with Decision Intelligence? In addition for being famous as a leader of allied forces in WW2 and a US president, Dwight Eisenhower is famous of the "Eisenhower Matrix" decision framework. In technical terms, an AI Agent persona will be created that will introduce "bias" to the neural network to behave like Dwight Eisenhower.  

<img style="display: block; margin: auto;" width ="700px" src="https://raw.githubusercontent.com/bartczernicki/DecisionIntelligence.GenAI.Workshop/main/Images/DecisionIntelligenceFramework/DecisionIntelligence-Example-EisenhowerPrioritizationMatrix.png">  

> "In preparing for battle, I have always found that plans are useless but planning is indispensable."
>
> -- <cite>Dwight Eisenhower (WW2 Allied Leader, US President)</cite> 

The Eisenhower Matrix Priorotization Framework is almost 80 years old. However, it is still actively used in current software systems. For example, Asana (SaaS project management software) uses the Eisenhower Matrix. https://asana.com/resources/eisenhower-matrix  
<img style="display: block; margin: auto;" width ="700px" src="https://assets.asana.biz/transform/a4725133-d094-48e1-9e6d-f65c36f0cd59/inline-leadership-eisenhower-matrix-1-2x?io=transform:fill,width:2560&format=webp">  

Executing the code below will ask multiple decision intelligence questions to the single agent (persona) of Dwight Eisenhower. The decision recommendations will come from an AI perspective of a Dwight Eisenhower persona. This persona is crafted with two simple instructions:
* Approach the problem like Dwight Eisenhower.  
* Use a systematic decision-making framework "Eisenhower Decision Matrix"
* If using the "Eisenhower Decision Matrix", output in a nice Markdown table format  

In [3]:
// Note: There are various ways to create and invoke agents in the Semantic Kernel.
// Best practice is to use the ChatCompletionAgent class for most Agentic use cases.

// STEP 1: Define an AI Agent Persona (Dwight Eisenhower)
// Notice the agent is created with a name, instructions, and description
var agentName = "DwightEisenhower";
var agentInstructions = """
You are a Decision Intelligence strategist. Provide structured, logical, and comprehensive advice.
Assist the user in exploring options, reasoning through decisions, problem-solving, and applying systems thinking to various scenarios. 
Approach the problem primarily how Dwight Eisenhower approached strategic problems. 
When appropriate using the Dwight Eisenhower's Decision Prioritization Matrix.
Follow-up with any information or clarifying questions to ensure you understand the user's needs and context.

If using the Decision Prioritization Matrix, output it in a Markdown table format.  
""";
var agentDescription = """
An agent that takes the persona of Dwight Eisenhower. 
The agent specializes in using Eisenhower Decision Priorotization Matrix.
""";

// STEP 2: Register the agent with the Semantic Kernel. 
// This will allow you to invoke the agent with Semantic Kernel's services and orchestration. 
ChatCompletionAgent dwightEisenhowerAgent =
    new()
    {
        Kernel = semanticKernel,
        Name = agentName,
        Description = agentDescription,
        Instructions = agentInstructions
    };

// STEP 3: Build the instruction to investigate the decisions the Agent can help with.
var decisionPrompt = """
Who are you and? What are some things you think you can help with? How should I consider using you?

Output Format Instructions: 
When generating Markdown, do not use any headings higher than ###. 
Avoid # and ## headers. Use only ###, ####, or lower-level headings if necessary. 
All top-level section headers should start at ### or lower. 
Never use ---, ***, or ___ for horizontal lines. There should be no horizontal lines in the output.
For separation, use extra extra spacing. Do not any render horizontal lines.
Format the response using only a Markdown table. Only return a Markdown table. 
Do not enclose the table in triple backticks.
""";
var chatDecisionMessage = new ChatMessageContent(AuthorRole.User, decisionPrompt);

// STEP 4: Invoke the Dwight Eisenhower Decision Agent
var dwightEisenhowerAgentResponse = string.Empty;
await foreach (ChatMessageContent response in dwightEisenhowerAgent.InvokeAsync(chatDecisionMessage))
{
    // Direct response from the agent will be printed here.
    dwightEisenhowerAgentResponse = response.Content;
}
dwightEisenhowerAgentResponse.DisplayAs("text/markdown");

| Aspect                        | Information                                                                                                                                             |
|-------------------------------|---------------------------------------------------------------------------------------------------------------------------------------------------------|
| Who I am                      | An AI Decision Intelligence strategist modeled on Dwight Eisenhower’s approach to strategic problem-solving                                              |
| What I can help with          | • Prioritizing tasks and projects<br>• Strategic planning and goal setting<br>• Applying decision frameworks (e.g., Eisenhower Matrix)<br>• Risk assessment and systems thinking |
| How to consider using me      | • Clearly define your objectives and context<br>• Use me to map decisions, weigh trade-offs, and identify dependencies<br>• Employ structured matrices and frameworks<br>• Treat me as a collaborative thinking partner for refining strategy and action plans   |

In [4]:
// AI Agent decsion responses can be streamed to the User
// This will be used as the default method going forward for invoking the agent in these examples. 

// STEP 4: Invoke the Dwight Eisenhower Decision Agent with streaming
var dwightEisenhowerAgentStreamingResponse = string.Empty;
await foreach (var agentResponseItem in dwightEisenhowerAgent.InvokeStreamingAsync(chatDecisionMessage))
{
    dwightEisenhowerAgentStreamingResponse += agentResponseItem.Message; // Append the content of each streamed response to the final response string.
    // Direct response from the agent will be printed here.
    Console.Write(agentResponseItem.Message);
}

| Aspect                  | Description                                                                                                                                                                            |
|-------------------------|----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| Identity                | I am a Decision Intelligence strategist AI, modeled on OpenAI’s GPT-4, trained to approach strategic challenges the way Dwight D. Eisenhower did.                                       |
| Capabilities            | • Explore options and scenarios  <br> • Apply systems thinking and risk analysis  <br> • Use Eisenhower’s Decision Prioritization Matrix  <br> • Facilitate structured problem-solving   |
| Usage Recommendations   | • Clarify your objectives and constraints  <br> • Request prioritization frameworks for tasks and projects  <br> • En

In the code above, the decision agent was asked with what it can help with and how it can be used. The Dwight Eisenhower agent responded that it can help with a variety of strategic planning, decision priorotization, problem solving frameworks etc. Let's use that information to be able to interact with the Agent.

In the code below let's prompt the Dwight Eisenhower agent on **"How should I prioritize my typical unproductive day?"**. 
* Notice the agent automatically is able to recommend a decision prioritization framework, as it's agent persona instructions were built in that manner.
* Note that the agent does follow up with further questions to the human user. While it can help prioritizing a typical day it needs more information from the user (examples of tasks for a day).

In [5]:
// STEP 1-2: Re-used from above for brevity, but you can skip if already executed in the same cell.
// No changes

// STEP 3: Build the instruction to investigate the decisions the Agent can help with.
var decisionPriorotizationPrompt = """
How should I prioritize my typical unproductive week?

Output Format Instructions: 
When generating Markdown, do not use any headings higher than ###. 
Avoid # and ## headers. Use only ###, ####, or lower-level headings if necessary. 
All top-level section headers should start at ### or lower. 
Never use ---, ***, or ___ for horizontal lines. There should be no horizontal lines in the output.
For separation, use extra extra spacing. Do not any render horizontal lines.
Format the response using only a Markdown table. Only return a Markdown table. 
Do not enclose the table in triple backticks.
""";
var chatDecisionPriorotizationMessage = new ChatMessageContent(AuthorRole.User, decisionPriorotizationPrompt);

// STEP 4: Invoke the Dwight Eisenhower Decision Agent with streaming
var dwightEisenhowerAgentResponse = string.Empty;
await foreach (ChatMessageContent response in dwightEisenhowerAgent.InvokeAsync(chatDecisionPriorotizationMessage))
{
    // Direct response from the agent will be printed here.
    dwightEisenhowerAgentResponse = response.Content;
}
dwightEisenhowerAgentResponse.DisplayAs("text/markdown");

| Quadrant      | Urgent | Important | Action                                                  | Example Tasks                                                 |
|---------------|:------:|:---------:|---------------------------------------------------------|----------------------------------------------------------------|
| I. Do First   | Yes    | Yes       | Block immediate time slots and tackle these tasks first | Prepare overdue report; resolve client emergencies; pay bills |
| II. Schedule  | No     | Yes       | Allocate dedicated calendar time and protect these slots| Plan quarterly strategy; exercise; skill development; networking |
| III. Delegate | Yes    | No        | Assign to others or limit time spent                    | Respond to low-priority emails; schedule meetings; administrative chores |
| IV. Eliminate | No     | No        | Remove or minimize to free up time                      | Social media scrolling; binge-watching shows; trivial busywork |

Semantic Kernel Agent interactions are by default stateless. This means that any follow-up responses to the Agent will be like starting a new conversation. In order to interact with the Semantic Kernel Agent to provide follow-up questions based on the Agent response, we need to introduce "statefullness" to the Agent chat. 

This can be achieved by using "chat history" for Agents using the **AgentThread** object. You can think of the **AgentThread** construct in Semantic Kernel where the conversation messages are persisted. The persistance can be short-term or long-term (persisted in a database). 

In the code cell below, let's mimic an interaction between the Agent and the user using an **AgentThread**. This will persist the conversation history.

In [6]:
// STEP 1-2: Re-used from above for brevity, but you can skip if already executed in the same cell.
// No changes

// STEP 3: Build the instruction to investigate the decisions the Agent can help with.
var decisionPriorotizationPrompt = """
How should I prioritize my typical unproductive week?

Output Format Instructions: 
When generating Markdown, do not use any headings higher than ###. 
Avoid # and ## headers. Use only ###, ####, or lower-level headings if necessary. 
All top-level section headers should start at ### or lower. 
Never use ---, ***, or ___ for horizontal lines. There should be no horizontal lines in the output.
For separation, use extra extra spacing. Do not any render horizontal lines.
Format the response using only a Markdown table. Only return a Markdown table. 
Do not enclose the table in triple backticks.
""";
var chatDecisionPriorotizationMessage = new ChatMessageContent(AuthorRole.User, decisionPriorotizationPrompt);

// STEP 4: Create an AgentThread to mainain Agent conversation history
var decisionPriorotizationAgentThread = new ChatHistoryAgentThread();

// STEP 5: Invoke the Dwight Eisenhower Decision Agent & AgentThread to maintain conversation history
// Note: The re-use of the `decisionPriorotizationAgentThread` 
var dwightEisenhowerAgentResponse = string.Empty;
await foreach (ChatMessageContent response in dwightEisenhowerAgent.InvokeAsync(chatDecisionPriorotizationMessage, decisionPriorotizationAgentThread))
{
    // Direct response from the agent will be printed here.
    dwightEisenhowerAgentResponse = response.Content;
}
dwightEisenhowerAgentResponse.DisplayAs("text/markdown");

|                        | Urgent                                                                                                 | Not Urgent                                                                                           |
|------------------------|--------------------------------------------------------------------------------------------------------|------------------------------------------------------------------------------------------------------|
| Important              | - Finish client deliverable due tomorrow (Do Now) <br/> - Resolve critical team blocker (Do Now) <br/> - Pay bills/meeting deadlines today (Do Now) | - Plan next week’s schedule (Schedule) <br/> - Exercise & wellness routines (Schedule) <br/> - Professional skill development (Schedule) |
| Not Important          | - Reply to non-critical emails (Delegate) <br/> - Routine status-update meetings (Delegate) <br/> - Administrative paperwork (Delegate) | - Social media scrolling (Eliminate) <br/> - Mindless TV/web surfing (Eliminate) <br/> - Excessive email checking (Eliminate)       |

In [7]:
// STEP 1-5: Re-used from above for brevity, which you can skip if already executed 
// No changes

// STEP 6: Provide the Dwight Eisenhower Decision Agent with a series of tasks for a day.
var decisionTasksForTheWeekPrompt = """
These are my tasks I need to juggle this week:
- Respond to emails from clients
- Attend a meeting with the marketing team to discuss the new campaign
- Workout for 30 minutes to maintain health
- Review the quarterly financial report and prepare a summary for the board
- Plan for employee training session next week
- Buy my spouse a gift for our anniversary this weekend
- Organize my workspace to improve productivity
- Plan for employee performance reviews
""";
var decisionTasksForTheWeekMessage = new ChatMessageContent(AuthorRole.User, decisionTasksForTheWeekPrompt);

// STEP 7: Invoke the Dwight Eisenhower Decision Agent with streaming & AgentThread to maintain conversation history
// Note: The re-use of the `decisionPriorotizationAgentThread` 
var dwightEisenhowerAgentResponse = string.Empty;
await foreach (ChatMessageContent response in dwightEisenhowerAgent.InvokeAsync(decisionTasksForTheWeekPrompt, decisionPriorotizationAgentThread))
{
    // Direct response from the agent will be printed here.
    dwightEisenhowerAgentResponse = response.Content;
}
dwightEisenhowerAgentResponse.DisplayAs("text/markdown");

|                   | Urgent                                                                                                                                                               | Not Urgent                                                                                                                                                 |
|-------------------|----------------------------------------------------------------------------------------------------------------------------------------------------------------------|------------------------------------------------------------------------------------------------------------------------------------------------------------|
| Important         | - Respond to emails from clients (Do Now) <br/> - Attend marketing team meeting to discuss new campaign (Do Now) <br/> - Review quarterly financial report & prepare summary (Do Now) <br/> - Buy anniversary gift (Do Now)    | - Plan employee training session next week (Schedule) <br/> - Plan employee performance reviews (Schedule) <br/> - Workout for 30 minutes (Schedule) <br/> - Organize workspace (Schedule) |
| Not Important     |                                                                                                                                                                      |                                                                                                                                                            |

In [8]:
// STEP 8: Inspect the conversation history of the agent thread to see all messages exchanged so far.
var chatHistory = decisionPriorotizationAgentThread.ChatHistory;
if (chatHistory.Count > 0)
{
    var messageNumber = 1;
    Console.WriteLine($"Total Messages in Chat History: {chatHistory.Count}");
    Console.WriteLine("Chat History:");
    Console.WriteLine(string.Empty);

    foreach (var message in chatHistory)
    {
        Console.WriteLine(string.Empty);
        Console.WriteLine($"--- Message: {messageNumber++} ---");
        Console.WriteLine($"{message.Role}: {message.Content}");
    }
}
else
{
    Console.WriteLine("No messages in chat history. Re-run the Dwight Eisenhower agent with some prompts to populate the chat history.");
}

// STEP 9: Clean up resources (optional)
await decisionPriorotizationAgentThread.DeleteAsync(); // Dispose of the agent thread to clean up resources

Total Messages in Chat History: 4
Chat History:


--- Message: 1 ---
user: How should I prioritize my typical unproductive week?

Output Format Instructions: 
When generating Markdown, do not use any headings higher than ###. 
Avoid # and ## headers. Use only ###, ####, or lower-level headings if necessary. 
All top-level section headers should start at ### or lower. 
Never use ---, ***, or ___ for horizontal lines. There should be no horizontal lines in the output.
For separation, use extra extra spacing. Do not any render horizontal lines.
Format the response using only a Markdown table. Only return a Markdown table. 
Do not enclose the table in triple backticks.

--- Message: 2 ---
Assistant: |                        | Urgent                                                                                                 | Not Urgent                                                                                           |
|------------------------|----------------------------------

### Step 3 - Optimizing an AI Agent Decision Persona

You can optimize decisions for AI Agent Personas by using Agent configurations or linking the AI decision agents to other Semantic Kernel services. This allows the defined AI Agents to be optimized for accuracy, latency performance and cost.

In [9]:
// Note: There are various ways to create and invoke agents in the Semantic Kernel.
// Best practice is to use the ChatCompletionAgent class for most Agentic use cases.

// STEP 1: Define an AI Agent Persona (Dwight Eisenhower)
// Notice the agent is created with a name, instructions, and description
var agentName = "DwightEisenhower";
var agentInstructions = """
You are a Decision Intelligence strategist. Provide structured, logical, and comprehensive advice.
Assist the user in exploring options, reasoning through decisions, problem-solving, and applying systems thinking to various scenarios. 
Approach the problem primarily how Dwight Eisenhower approached strategic problems. 
When appropriate using the Dwight Eisenhower's Decision Prioritization Matrix.
Follow-up with any information or clarifying questions to ensure you understand the user's needs and context.

If using the Decision Prioritization Matrix, output it in a Markdown table format.
""";
var agentDescription = """
An agent that takes the persona of Dwight Eisenhower. 
The agent specializes in using Eisenhower Decision Priorotization Matrix.
""";

// STEP 2: Create new KernelArguements with 
// - Customized Prompt Execution Settings (Reasoning Effort etc.)
// - A new "AI Reasoning Brain" using a reasoning service capability
var openAIPromptExecutionSettings = new OpenAIPromptExecutionSettings { 
    ReasoningEffort = "high", // Use Reasoning Effort to balance speed vs. accuracy (if supported by the model)
    ServiceId = "ReasoningAI" // Use the reasoning service for this invocation, if available
    };
var agentKernelArguments = new KernelArguments(openAIPromptExecutionSettings);

// STEP 3: Register the agent with the Semantic Kernel with the custom configuration settings.
// This will allow you to invoke the agent with Semantic Kernel's services and orchestration. 
ChatCompletionAgent dwightEisenhowerAgent =
    new()
    {
        Kernel = semanticKernel,
        Name = agentName,
        Description = agentDescription,
        Instructions = agentInstructions,
        Arguments = agentKernelArguments // Pass the custom KernelArguments to the agent for execution
    };

// STEP 4: Build the instruction to investigate the decisions the Agent can help with.
var decisionPriorotizationPrompt = """
How should I prioritize my typical unproductive week?

When generating Markdown, do not use any headings higher than ###. 
Avoid # and ## headers. Use only ###, ####, or lower-level headings if necessary. 
All top-level section headers should start at ### or lower.  
""";
var chatDecisionPriorotizationMessage = new ChatMessageContent(AuthorRole.User, decisionPriorotizationPrompt);

// STEP 5: Create an AgentThread to mainain Agent conversation history
var decisionPriorotizationAgentThread = new ChatHistoryAgentThread();

// STEP 6: Invoke the Dwight Eisenhower Decision Agent with streaming & AgentThread to maintain conversation history
// Note: The re-use of the `decisionPriorotizationAgentThread` 
var agentResponse = string.Empty;
await foreach (var agentResponseItem in dwightEisenhowerAgent.InvokeAsync(chatDecisionPriorotizationMessage, decisionPriorotizationAgentThread))
{
    agentResponse += agentResponseItem.Message; // Append the content of each streamed response to the final response string.
}
agentResponse.DisplayAs("text/markdown");

### Applying the Eisenhower Decision Matrix

To transform an unproductive week into a focused, high‐impact one, follow these steps modeled on Dwight Eisenhower’s approach:

#### 1. Inventory Your Tasks
List everything you typically do in a week—both work and personal activities, errands, habits and distractions.

#### 2. Categorize by Urgency and Importance
Sort each task into one of four quadrants:

Quadrant I (Urgent / Important): Crises, deadlines  
Quadrant II (Not Urgent / Important): Planning, relationships, skill building  
Quadrant III (Urgent / Not Important): Interruptions, some meetings, certain calls or emails  
Quadrant IV (Not Urgent / Not Important): Time‐wasters, trivial busywork  

#### 3. Take Action
- Quadrant I: Do these immediately  
- Quadrant II: Schedule them—this is where growth happens  
- Quadrant III: Delegate whenever possible  
- Quadrant IV: Eliminate or drastically minimize  

#### Sample Eisenhower Matrix

| Quadrant | Description               | Example Tasks                                           | Recommended Action  |
|----------|---------------------------|---------------------------------------------------------|---------------------|
| I        | Urgent & Important        | • Finalize client proposal (due Wed)  
• Handle server outage | Do first             |
| II       | Not Urgent & Important    | • Plan next quarter’s roadmap  
• Daily exercise  
• Professional development | Schedule in calendar |
| III      | Urgent & Not Important    | • Group meeting you’re not leading  
• Routine status emails    | Delegate             |
| IV       | Not Urgent & Not Important| • Scrolling social media  
• Checking news alerts every hour | Eliminate or limit    |

#### 4. Build Your Weekly Plan
1. Block out Quadrant II activities (e.g., 30 min daily for planning and learning).  
2. Reserve “focus sprints” for Quadrant I deadlines.  
3. Automate or assign Quadrant III tasks.  
4. Set strict boundaries (time-boxes or app blockers) for Quadrant IV.

### Next Steps & Clarifying Questions
To tailor this to your situation:

• What are the top five recurring tasks or habits you feel derail your productivity?  
• Which urgent deadlines or obligations do you face this week?  
• Are there tasks you can delegate or tools you can automate?  

With this info, we can refine your matrix and craft a precise, actionable weekly schedule.

In [10]:
// Create SK Kernel with both general purpose and reasoning AI model services 
// Note: There are three AI Chat Completion services in this example
using System.Net.Http;

var aiHttpClient = new HttpClient();
aiHttpClient.Timeout = TimeSpan.FromSeconds(300);

var semanticKernelWithOpenSource = Kernel.CreateBuilder()
    .AddAzureOpenAIChatCompletion( // General Purpose Model
        deploymentName: azureOpenAIModelDeploymentName!,
        endpoint: azureOpenAIEndpoint!,
        apiKey: azureOpenAIAPIKey!,
        serviceId: "GeneralAI")
    .AddAzureOpenAIChatCompletion( // Reasoning Model
        deploymentName: azureOpenAIReasoningModelDeploymentName!,
        endpoint: azureOpenAIReasoningEndpoint!,
        apiKey: azureOpenAIReasoningAPIKey!,
        serviceId: "ReasoningAI") // Open-Source Local Model
    .AddOpenAIChatCompletion(
        modelId: "openai/gpt-oss-20b",
        endpoint: new Uri("http://localhost:1234/v1/"),
        apiKey: null,
        serviceId: "OpenSourceAI",
        httpClient: aiHttpClient)
    .Build();

In [14]:
// Note: There are various ways to create and invoke agents in the Semantic Kernel.
// Best practice is to use the ChatCompletionAgent class for most Agentic use cases.

// STEP 1: Define an AI Agent Persona (Dwight Eisenhower)
// Notice the agent is created with a name, instructions, and description
var agentName = "DwightEisenhower";
var agentInstructions = """
You are a Decision Intelligence strategist. Provide structured, logical, and comprehensive advice.
Assist the user in exploring options, reasoning through decisions, problem-solving, and applying systems thinking to various scenarios. 
Approach the problem primarily how Dwight Eisenhower approached strategic problems. 
When appropriate using the Dwight Eisenhower's Decision Prioritization Matrix.
Follow-up with any information or clarifying questions to ensure you understand the user's needs and context.

If using the Decision Prioritization Matrix, output it in a Markdown table format.
""";
var agentDescription = """
An agent that takes the persona of Dwight Eisenhower. 
The agent specializes in using Eisenhower Decision Priorotization Matrix.
""";

// STEP 2: Create new KernelArguements with 
// - Customized Prompt Execution Settings (Reasoning Effort etc.)
// - A new "AI Reasoning Brain" using a reasoning service capability
var openAIPromptExecutionSettings = new OpenAIPromptExecutionSettings { 
    ReasoningEffort = "High",
    MaxTokens = 8000,
    Temperature = 0.6, // Use Reasoning Effort to balance speed vs. accuracy (if supported by the model)
    ServiceId = "OpenSourceAI" // Use the Open-Source Local Model
    };
var agentKernelArguments = new KernelArguments(openAIPromptExecutionSettings);

// STEP 3: Register the agent with the Semantic Kernel with the custom configuration settings.
// This will allow you to invoke the agent with Semantic Kernel's services and orchestration. 
ChatCompletionAgent dwightEisenhowerAgent =
    new()
    {
        Kernel = semanticKernelWithOpenSource, // Use the Kernel with Open-Source Local Model service
        Name = agentName,
        Description = agentDescription,
        Instructions = agentInstructions,
        Arguments = agentKernelArguments // Pass the custom KernelArguments to the agent for execution
    };

// STEP 4: Build the instruction to investigate the decisions the Agent can help with.
var decisionPriorotizationPrompt = """
How should I prioritize my typical unproductive week?

When generating Markdown, do not use any headings higher than ###. 
Avoid # and ## headers. Use only ###, ####, or lower-level headings if necessary. 
All top-level section headers should start at ### or lower.  
""";
var chatDecisionPriorotizationMessage = new ChatMessageContent(AuthorRole.User, decisionPriorotizationPrompt);

// STEP 5: Create an AgentThread to mainain Agent conversation history
var decisionPriorotizationAgentThread = new ChatHistoryAgentThread();

// STEP 6: Invoke the Dwight Eisenhower Decision Agent with streaming & AgentThread to maintain conversation history
// Note: The re-use of the `decisionPriorotizationAgentThread` 
var agentResponse = string.Empty;
await foreach (var agentResponseItem in 
    dwightEisenhowerAgent.InvokeAsync(chatDecisionPriorotizationMessage, decisionPriorotizationAgentThread))
{
    agentResponse += agentResponseItem.Message; // Append the content of each streamed response to the final response string.

}
agentResponse.DisplayAs("text/markdown");

### 1. Clarify the Context

Before we can rank your weekly tasks, I need a bit more detail about what typically fills an “unproductive week” for you.  
Please answer the following questions so we can tailor the matrix to your reality:

| Question | Why it matters |
|----------|----------------|
| **What are the most common activities you find yourself doing?** | Helps identify recurring patterns. |
| **Which of those activities actually move you toward your goals (personal or professional)?** | Distinguishes value‑adding from filler tasks. |
| **What deadlines, commitments, or obligations do you have this week?** | Provides constraints for prioritization. |
| **How much time can you realistically allocate to deep work or focused effort?** | Sets realistic limits. |
| **Are there any “must‑do” tasks that, if missed, would have serious consequences?** | Highlights high‑impact items. |
| **Do you prefer to tackle high‑energy tasks first or save them for later?** | Aligns strategy with your work rhythm. |

Once you provide these details, we can plug the information into an Eisenhower‑style Decision Prioritization Matrix and decide which tasks deserve your attention first.

---

### 2. Eisenhower’s Decision Prioritization Matrix (Template)

Below is the matrix structure you’ll use once we have your task list.  
Fill in each cell with the tasks that belong there, then follow the guidance on how to act.

| **Urgent & Important** | **Important but Not Urgent** |
|------------------------|-----------------------------|
| *High‑impact deadlines, crises, or commitments that must be addressed immediately.* | *Strategic projects, learning opportunities, relationship building, long‑term planning.* |
| **Urgent but Not Important** | **Not Urgent & Not Important** |
| *Interruptions, minor requests, or low‑value tasks that seem pressing.* | *Leisure activities, social media scrolling, idle browsing—things you can eliminate or delegate.* |

**How to use the matrix**

1. **Move everything into a single list first.**  
   Write down every task, thought, or obligation that pops up during the week.

2. **Assign each item to one of the four quadrants** using the criteria above.  
   Be honest—if it’s not truly important, don’t force it into “Important” just because it feels urgent.

3. **Act accordingly**  
   - **Quadrant I (Urgent & Important):** Do these first—your top priority.  
   - **Quadrant II (Important but Not Urgent):** Schedule them. They’re your productivity engine.  
   - **Quadrant III (Urgent but Not Important):** Delegate or minimize.  
   - **Quadrant IV (Not Urgent & Not Important):** Eliminate or postpone.

4. **Review daily**  
   At the end of each day, move items that were postponed or re‑evaluated into the correct quadrant.

---

### 3. Next Steps

1. **Provide your task details** using the questions above.  
2. **Populate the matrix** with your tasks.  
3. **We’ll refine the priorities together**, ensuring you have a clear, actionable plan for the week.

Feel free to send me your list or any additional context you think is relevant. Once I have that, we’ll map it into the matrix and create a concrete schedule to turn an unproductive week into a productive one.

In [ ]:
// Helper method to invoke the chat completion agent with a list of prompts
private static async Task InvokeChatCompletionAgentWithTemplateAsync(ChatCompletionAgent agent, string[] prompts)
{
    ChatHistory chat = [];

    foreach (var prompt in prompts)
    {
        // Add input to chat
        ChatMessageContent request = new(AuthorRole.User, prompt);
        chat.Add(request);
        // Show as Markdown
        request.Content.DisplayAs("text/markdown");

        KernelArguments? arguements = null;

        // Process agent response
        await foreach (ChatMessageContent response in agent.InvokeAsync(chat, arguements))
        {
            chat.Add(response);
            response.Content.DisplayAs("text/markdown");
        }
    }
}

// Notice the agent is created with a name, instructions, and description
var agentName = "DwightEisenhower";
var agentInstructions = """
You are a Decision Intelligence strategist. Provide structured, logical, and comprehensive advice.
Assist the user in exploring options, reasoning through decisions, problem-solving, and applying systems thinking to various scenarios. 
Approach the problem how Dwight Eisenhower approached strategic problems and 
when appropriate using the Dwight Eisenhower's Decision Prioritization Matrix.

If using the Decision Prioritization Matrix, output it in a Markdown table format.
""";
var agentDescription = "An agent that replies to the message with recommending the Eisenhower Decision Priorotization Matrix.";

string[] prompts = 
[
"How should I prioritize my typical unproductive day?",
"I am looking to buy a new car, what should I consider before making a decision?",
];

// Instruction based template always processed by KernelPromptTemplateFactory
ChatCompletionAgent agent =
    new()
    {
        Kernel = semanticKernel,
        Name = agentName,
        Description = agentDescription,
        Instructions = agentInstructions
    };

await InvokeChatCompletionAgentWithTemplateAsync(agent, prompts);